In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
import pygeodesic.geodesic as geodesic
import pyvista as pv
import vtk
from vtk_helpers import *
from scipy.spatial import Delaunay, ConvexHull
import potpourri3d as pp3d
from tqdm import tqdm
import time

In [ ]:
root = "./Surface_Defects_pcd_extend_2000_geod_estnorm_noise0001/"
folders = ['normal','raised', 'dent', 'crack']
N = 50
n = 2000
for fold in folders:
      
    cate = root + fold + '/'

    for i in tqdm(range(N)):
        file = cate + fold + str(i+1) + '.txt'
        pts = np.loadtxt(file, delimiter = ',')[:,:3]
        
        faces = Delaunay(pts)
        mesh_1 = np.copy(faces.simplices)[:,:-1]
        mesh_2 = np.copy(faces.simplices)[:,1:]
        mesh = np.concatenate((mesh_1, mesh_2))
        solver = pp3d.MeshHeatMethodDistanceSolver(pts,mesh,t_coef=0.01,use_robust=True)
        pairwise_dist = np.zeros((n,n))
        dis_filename = '.' + file.split('.')[1] + '_geodesic_dis.npz'
#         print(dis_filename)
        for i in range(n):
            pairwise_dist[i,:] = solver.compute_distance(i)
        
        
        np.savez(dis_filename, data = pairwise_dist)